# Dog vs Cat Classification

Two models are compared here. A normal CNN model and a fine tuned transfer learning model(VGG16 model). The dataset details are provided in the readme file. 

Importing packages (Tensorflow is the preferred library in this project)

In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

In [4]:
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore',category=FutureWarning)
%matplotlib inline

In [6]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num of GPUs available: ",len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0],True)

Num of GPUs available:  1


# Organize into train, validation and test folders


In [7]:
os.chdir('dogsVcats_train')

In [ ]:
os.getcwd()

Creating sub folders to organize the raw images so that training becomes easy

In [8]:
if os.path.isdir('train/dog') is False:
    os.makedirs('train/dog')
    os.makedirs('train/cat')
    os.makedirs('test/dog')
    os.makedirs('test/cat')
    os.makedirs('valid/dog')
    os.makedirs('valid/cat')

# Move randomly chosen images to directories

In [25]:
for c in random.sample(glob.glob('cat*'),500):
    shutil.move(c,'train/cat')
for c in random.sample(glob.glob('dog*'),500):
    shutil.move(c,'train/dog')
for c in random.sample(glob.glob('cat*'),100):
    shutil.move(c,'test/cat')
for c in random.sample(glob.glob('cat*'),200):
    shutil.move(c,'valid/cat')
for c in random.sample(glob.glob('dog*'),100):
    shutil.move(c,'test/dog')
for c in random.sample(glob.glob('dog*'),200):
    shutil.move(c,'valid/dog')
os.chdir(os.path.dirname(os.getcwd()))

In [28]:
train_path = 'dogsVcats_train/train'
test_path = 'dogsVcats_train/test'
valid_path = 'dogsVcats_train/valid'

The images from each folder and grouped into batches of size 10. (If OOM error occurs, reduce the batch size)

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=train_path, target_size=(224,224), classes=['cat', 'dog'],batch_size=10)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=valid_path, target_size=(224,224), classes=['cat', 'dog'],batch_size=10)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\
    .flow_from_directory(directory=test_path, target_size=(224,224), classes=['cat', 'dog'],batch_size=10)

In [30]:
imgs,labels = next(train_batches)

In [31]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

Plotting few images

In [ ]:
plotImages(imgs)
print(labels)

# Model 1: Regular CNN Model 
(Convolution and Max pool layers)

In [33]:
model = Sequential([
    Conv2D(filters=32,kernel_size=(3,3),activation='relu',padding='same',input_shape=(224,224,3)),
    MaxPool2D(pool_size=(2,2),strides=2),
    Conv2D(filters=64,kernel_size=(3,3),activation='relu',padding='same'),
    MaxPool2D(pool_size=(2,2),strides=2),
    Flatten(),
    Dense(units=2, activation='softmax'),
])

In [34]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 200704)            0         
                                                                 
 dense (Dense)               (None, 2)                 4

In [35]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

Epochs refers to the number of times the models looks at the whole dataset

Verbose controls the amount of information that will be displayed during training

steps in each epoch = Total no. of training images/batch size

In [ ]:
model.fit(x=train_batches,
    validation_data=valid_batches,
    epochs=2,
    verbose=2
)

# Prediction of model 1

In [ ]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)
np.round(predictions)

# 0 means cat and 1 means dog

In [38]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))

In [39]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [42]:
test_batches.class_indices

{'cat': 0, 'dog': 1}

In [ ]:
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

# Model 2: VGG model

In [ ]:
vgg16_model = tf.keras.applications.vgg16.VGG16()
vgg16_model.summary()

In [45]:
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

All the layers in VGG16 are not trainable. An additional dense layer is added whose weights are trainable

In [46]:
for layer in model.layers:
    layer.trainable = False

In [47]:
model.add(Dense(units=2, activation='softmax'))
# 2 units, 1 for rach class(cat or dog)

In [48]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [50]:
model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(x=train_batches,
    validation_data=valid_batches,
    epochs=2,
    verbose=2
)

# Prediction of Model 2 

In [52]:
predictions = model.predict(x=test_batches, steps=len(test_batches), verbose=0)

In [ ]:
cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(predictions, axis=-1))
cm_plot_labels = ['cat','dog']
plot_confusion_matrix(cm=cm, classes=cm_plot_labels, title='Confusion Matrix')

Alter the hyper-parameters(no. of epochs, batch size) to get best results

Calculate the accuracy and you will find the accuracy of VGG16 model to be high enough though it has fewer trainable parameters (Magic of transfer learning models)

Write some code to compare accuracy, F1 score, Recall and precision